[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Mix-of-Show-colab/blob/main/Mix_of_Show_fused_multi_real_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Mix-of-Show
%cd /content/Mix-of-Show

!pip install -q diffusers==0.20.2 transformers accelerate einops omegaconf
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl

!git clone https://huggingface.co/ckpt/MoS-potter-hermione-thanos-chilloutmix

%cd /content/Mix-of-Show

fused_model="/content/Mix-of-Show/MoS-potter-hermione-thanos-chilloutmix"
expdir="/content/Mix-of-Show/potter+hermione+thanos_chilloutmix"

keypose_condition='/content/Mix-of-Show/datasets/validation_spatial_condition/multi-characters/anime_pose_2x/hina_tezuka_kario_2x.png'
keypose_adaptor_weight=1.0
sketch_condition=''
sketch_adaptor_weight=1.0

context_prompt='two boys and a girl are standing near a forest'
context_neg_prompt='longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

region1_prompt='[a <potter1> <potter2>, standing near a forest]'
region1_neg_prompt="[${context_neg_prompt}]"
region1='[12, 36, 1024, 600]'

region2_prompt='[a <hermione1> <hermione2>, standing near a forest]'
region2_neg_prompt="[${context_neg_prompt}]"
region2='[18, 696, 1024, 1180]'

region5_prompt='[a <thanos1> <thanos2>, standing near a forest]'
region5_neg_prompt="[${context_neg_prompt}]"
region5='[142, 1259, 1024, 1956]'

prompt_rewrite=f"{region1_prompt}-*-{region1_neg_prompt}-*-{region1}|{region2_prompt}-*-{region2_neg_prompt}-*-{region2}|{region5_prompt}-*-{region5_neg_prompt}-*-{region5}"

print(prompt_rewrite)

!python regionally_controlable_sampling.py \
  --pretrained_model="{fused_model}" \
  --sketch_adaptor_weight="{sketch_adaptor_weight}" \
  --sketch_condition="{sketch_condition}" \
  --keypose_adaptor_weight="{keypose_adaptor_weight}" \
  --keypose_condition="{keypose_condition}" \
  --save_dir="{expdir}" \
  --prompt="{context_prompt}" \
  --negative_prompt="{context_neg_prompt}" \
  --prompt_rewrite="{prompt_rewrite}" \
  --suffix="baseline" \
  --seed=19